# 📊 Options IV Screener (Mini)

**Purpose**: Quick-scan IV scoring and ranking from the equity screener. No strategy details, pricing, or sizing—just pure IV opportunity scoring.

**What this produces**:
1. Screened tickers ranked by IV opportunity score
2. ATM IV, HV comparison, and term structure slope
3. Composite grade combining fundamentals + IV metrics
4. Clean visualizations for quick decision-making

In [12]:
import os
import time
import warnings
import math
from datetime import datetime
from typing import Optional

import numpy as np
import pandas as pd
import yfinance as yf
from yfinance import EquityQuery

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

try:
    from IPython.display import display, Markdown
except ImportError:
    def display(x):
        print(x)
    class Markdown(str):
        pass

try:
    import jinja2
    HAS_JINJA = True
except Exception:
    HAS_JINJA = False

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 40)
pd.set_option("display.width", 180)

---
## ⚙️ Configuration

In [13]:
# Universe selection
USE_SCREEN = os.getenv("USE_SCREEN", "1") == "1"
TICKER_OVERRIDE = [
    t.strip().upper() for t in os.getenv("TICKER_OVERRIDE", "").split(",") if t.strip()
]

SCREEN_PARAMS = dict(
    max_price=300.0,
    min_market_cap=2_000_000_000,
    min_roe=0.12,
    min_rev_growth=0.05,
    max_pe=40.0,
    max_ps=10.0,
    min_beta=1.0,
    min_inst_held=0.40,
    size=40,
    sort_by="eodvolume",
)

# IV Analysis
TARGET_DTE = 45
MAX_TERM_DTE = 120
TERM_STRUCTURE_SAMPLE = 2
HV_WINDOW = 30

# Trend filtering
TREND_FILTER = True
MIN_TREND_SCORE = 0.34
MA_SHORT = 50
MA_LONG = 200
MA_SLOPE_LOOKBACK = 20
TREND_PERIOD = "1y"

# Rate limiting
RATE_LIMIT_SLEEP = 0.3

# Scoring weights
FUNDAMENTAL_WEIGHT = 0.40
IV_WEIGHT = 0.60

FUND_WEIGHTS = {
    "roe": 0.25,
    "rev_growth": 0.25,
    "profit_margin": 0.20,
    "debt_to_equity": 0.15,
    "pe": 0.15,
}

IV_WEIGHTS = {
    "atm_iv": 0.35,
    "iv_hv_ratio": 0.30,
    "term_slope": 0.20,
    "iv_rank": 0.15,
}

TOP_N = 15
MAX_TICKERS = 20

# Plotly
pio.renderers.default = "notebook_connected"

display(Markdown(f"**Config**: Screen={USE_SCREEN}, Target DTE={TARGET_DTE}, Max Tickers={MAX_TICKERS}"))

**Config**: Screen=True, Target DTE=45, Max Tickers=20

---
## 🔧 Core Functions

In [14]:
def safe_float(value):
    try:
        if value is None:
            return np.nan
        return float(value)
    except Exception:
        return np.nan


def display_table(df, caption=None, format_dict=None):
    if HAS_JINJA:
        styler = df.style
        if format_dict:
            styler = styler.format(format_dict)
        if caption:
            styler = styler.set_caption(caption)
        display(styler)
    else:
        if caption:
            display(Markdown(f"**{caption}**"))
        display(df)


def assign_grade(score):
    if score is None or pd.isna(score):
        return "N/A"
    if score >= 90:
        return "A+"
    if score >= 80:
        return "A"
    if score >= 70:
        return "B"
    if score >= 60:
        return "C"
    if score >= 50:
        return "D"
    return "F"


def screen_for_candidates(
    max_price: float = 300.0,
    min_market_cap: float = 2_000_000_000,
    min_roe: float = 0.12,
    min_rev_growth: float = 0.05,
    max_pe: float = 40.0,
    max_ps: float = 10.0,
    min_beta: float = 1.0,
    min_inst_held: float = 0.40,
    size: int = 50,
    sort_by: str = "eodvolume",
) -> list[str]:
    sectors = [
        "Communication Services", "Consumer Cyclical", "Consumer Defensive",
        "Financial Services", "Healthcare", "Industrials", "Technology",
    ]
    filters = [
        EquityQuery("eq", ["region", "us"]),
        EquityQuery("is-in", ["exchange", "NMS", "NYQ"]),
        EquityQuery("btwn", ["intradaymarketcap", min_market_cap, 4_000_000_000_000]),
        EquityQuery("btwn", ["intradayprice", 10, max_price]),
        EquityQuery("btwn", ["peratio.lasttwelvemonths", 0, max_pe]),
        EquityQuery("lt", ["lastclosemarketcaptotalrevenue.lasttwelvemonths", max_ps]),
        EquityQuery("gte", ["returnontotalcapital.lasttwelvemonths", min_roe]),
        EquityQuery("gte", ["returnonequity.lasttwelvemonths", min_roe]),
        EquityQuery("gte", ["totalrevenues1yrgrowth.lasttwelvemonths", min_rev_growth]),
        EquityQuery("gte", ["pctheldinst", min_inst_held]),
        EquityQuery("gte", ["beta", min_beta]),
        EquityQuery("is-in", ["sector"] + sectors),
    ]
    q = EquityQuery("and", filters)
    resp = yf.screen(q, size=size, sortField=sort_by, sortAsc=False)
    quotes = []
    if resp:
        if "quotes" in resp:
            quotes = resp.get("quotes", [])
        elif "finance" in resp:
            result = resp.get("finance", {}).get("result", [])
            if result:
                quotes = result[0].get("quotes", [])
    return [row.get("symbol") for row in quotes if row.get("symbol")]


def get_spot(ticker: str) -> Optional[float]:
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        hist = t.history(period="1d")
        if not hist.empty and "Close" in hist.columns:
            return float(hist["Close"].iloc[-1])
        return None
    except Exception:
        return None


def fetch_fundamentals(ticker: str) -> dict:
    t = yf.Ticker(ticker)
    time.sleep(RATE_LIMIT_SLEEP)
    try:
        info = t.info or {}
    except Exception:
        info = {}
    return {
        "ticker": ticker,
        "sector": info.get("sector"),
        "market_cap": safe_float(info.get("marketCap")),
        "pe": safe_float(info.get("trailingPE") or info.get("forwardPE")),
        "roe": safe_float(info.get("returnOnEquity")),
        "rev_growth": safe_float(info.get("revenueGrowth")),
        "profit_margin": safe_float(info.get("profitMargins")),
        "debt_to_equity": safe_float(info.get("debtToEquity")),
    }


def get_expirations(ticker: str) -> list[tuple[str, int]]:
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        exp_dates = t.options
        if not exp_dates:
            return []
        today = datetime.now().date()
        result = []
        for exp_str in exp_dates:
            try:
                exp_date = datetime.strptime(exp_str, "%Y-%m-%d").date()
                dte = (exp_date - today).days
                if dte > 0:
                    result.append((exp_str, dte))
            except ValueError:
                continue
        return sorted(result, key=lambda x: x[1])
    except Exception:
        return []


def fetch_chain(ticker: str, exp_date_str: str, spot: float):
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        chain = t.option_chain(exp_date_str)
        return chain.calls, chain.puts
    except Exception:
        return pd.DataFrame(), pd.DataFrame()


def compute_atm_iv(calls: pd.DataFrame, puts: pd.DataFrame, spot: float) -> Optional[float]:
    atm_ivs = []
    for df in [calls, puts]:
        if not df.empty and "impliedVolatility" in df.columns:
            valid = df[df["impliedVolatility"].notna() & (df["impliedVolatility"] > 0)]
            if not valid.empty:
                idx = (valid["strike"] - spot).abs().idxmin()
                atm_ivs.append(valid.loc[idx, "impliedVolatility"])
    return float(np.mean(atm_ivs)) if atm_ivs else None


def compute_hv(ticker: str, window: int = 30) -> Optional[float]:
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        hist = t.history(period="6mo")
        if hist.empty or "Close" not in hist.columns:
            return None
        closes = hist["Close"].dropna()
        if len(closes) < window:
            return None
        returns = np.log(closes / closes.shift(1)).dropna()
        return float(returns.iloc[-window:].std() * math.sqrt(252))
    except Exception:
        return None


def compute_trend_score(ticker: str) -> dict:
    default = {"trend_score": None, "trend_label": "N/A"}
    try:
        t = yf.Ticker(ticker)
        time.sleep(RATE_LIMIT_SLEEP)
        hist = t.history(period=TREND_PERIOD)
        if hist.empty or "Close" not in hist.columns:
            return default
        closes = hist["Close"].dropna()
        if len(closes) < MA_LONG:
            return default
        price = float(closes.iloc[-1])
        ma_short = float(closes.rolling(MA_SHORT).mean().iloc[-1])
        ma_long = float(closes.rolling(MA_LONG).mean().iloc[-1])
        ma_short_prev = float(closes.rolling(MA_SHORT).mean().iloc[-(MA_SLOPE_LOOKBACK + 1)])
        ma_slope = (ma_short - ma_short_prev) / ma_short_prev if ma_short_prev else 0

        flags = [
            1 if price > ma_long else 0,
            1 if ma_short > ma_long else 0,
            1 if ma_slope > 0 else 0,
        ]
        score = float(np.mean(flags))
        label = "Up" if score >= 0.67 else ("Down" if score <= 0.33 else "Side")
        return {"trend_score": score, "trend_label": label}
    except Exception:
        return default


def compute_term_slope(term_data: list[dict]) -> Optional[float]:
    if len(term_data) < 2:
        return None
    sorted_data = sorted(term_data, key=lambda x: x["dte"])
    near, far = sorted_data[0], sorted_data[-1]
    denom = far["dte"] - near["dte"]
    if denom == 0:
        return None
    return (far["atm_iv"] - near["atm_iv"]) / denom


def score_series(series: pd.Series, higher_better: bool = True) -> pd.Series:
    s = pd.to_numeric(series, errors="coerce")
    ranks = s.rank(pct=True)
    if not higher_better:
        ranks = 1 - ranks
    return ranks.fillna(0.5)

---
## 🔍 Run Screener

In [15]:
display(Markdown("# 📊 IV Screener Results"))
display(Markdown(f"**Run Date**: {datetime.now().strftime('%Y-%m-%d %H:%M')}"))

# Get tickers
if USE_SCREEN:
    TICKERS = screen_for_candidates(**SCREEN_PARAMS)
elif TICKER_OVERRIDE:
    TICKERS = list(TICKER_OVERRIDE)
else:
    TICKERS = []

if MAX_TICKERS:
    TICKERS = TICKERS[:MAX_TICKERS]

display(Markdown(f"**Tickers Loaded**: {len(TICKERS)}"))

if not TICKERS:
    display(Markdown("⚠️ No tickers loaded. Check USE_SCREEN or TICKER_OVERRIDE."))

# 📊 IV Screener Results

**Run Date**: 2026-02-06 13:19

**Tickers Loaded**: 20

In [16]:
# Fetch data
rows = []
skipped = []

for i, ticker in enumerate(TICKERS):
    if (i + 1) % 5 == 0:
        display(Markdown(f"Processing {i+1}/{len(TICKERS)}: **{ticker}**"))

    spot = get_spot(ticker)
    if spot is None:
        skipped.append((ticker, "no_spot"))
        continue

    # Fundamentals + trend
    fund = fetch_fundamentals(ticker)
    trend = compute_trend_score(ticker)

    # Trend filter
    if TREND_FILTER and (trend["trend_score"] or 0) < MIN_TREND_SCORE:
        skipped.append((ticker, "trend_filter"))
        continue

    # Options data
    expirations = get_expirations(ticker)
    if not expirations:
        skipped.append((ticker, "no_options"))
        continue

    # Find target expiration
    target_exp = min(expirations, key=lambda x: abs(x[1] - TARGET_DTE))
    exp_date, dte = target_exp

    calls, puts = fetch_chain(ticker, exp_date, spot)
    atm_iv = compute_atm_iv(calls, puts, spot)

    if atm_iv is None:
        skipped.append((ticker, "no_iv"))
        continue

    # HV
    hv = compute_hv(ticker, HV_WINDOW)
    iv_hv_ratio = atm_iv / hv if hv and hv > 0 else None

    # Term structure
    term_data = []
    term_exps = [e for e in expirations if e[1] <= MAX_TERM_DTE][::TERM_STRUCTURE_SAMPLE]
    for te, td in term_exps[:4]:
        tc, tp = fetch_chain(ticker, te, spot)
        tiv = compute_atm_iv(tc, tp, spot)
        if tiv:
            term_data.append({"dte": td, "atm_iv": tiv})

    term_slope = compute_term_slope(term_data)

    rows.append({
        "ticker": ticker,
        "spot": spot,
        "sector": fund.get("sector"),
        "market_cap": fund.get("market_cap"),
        "roe": fund.get("roe"),
        "rev_growth": fund.get("rev_growth"),
        "profit_margin": fund.get("profit_margin"),
        "debt_to_equity": fund.get("debt_to_equity"),
        "pe": fund.get("pe"),
        "trend_score": trend.get("trend_score"),
        "trend_label": trend.get("trend_label"),
        "dte": dte,
        "atm_iv": atm_iv,
        "hv_30": hv,
        "iv_hv_ratio": iv_hv_ratio,
        "term_slope": term_slope,
    })

display(Markdown("---"))
display(Markdown(f"✅ **Processed**: {len(rows)} | ⏭️ **Skipped**: {len(skipped)}"))

Processing 5/20: **PYPL**

Processing 10/20: **CRM**

Processing 15/20: **ASX**

Processing 20/20: **XYZ**

---

✅ **Processed**: 8 | ⏭️ **Skipped**: 12

---
## 🏆 Scoring & Ranking

In [17]:
df = pd.DataFrame(rows)

if df.empty:
    display(Markdown("⚠️ No data collected. Check tickers and options availability."))
else:
    # Fundamental score
    fund_score = (
        score_series(df["roe"], higher_better=True) * FUND_WEIGHTS["roe"] +
        score_series(df["rev_growth"], higher_better=True) * FUND_WEIGHTS["rev_growth"] +
        score_series(df["profit_margin"], higher_better=True) * FUND_WEIGHTS["profit_margin"] +
        score_series(df["debt_to_equity"], higher_better=False) * FUND_WEIGHTS["debt_to_equity"] +
        score_series(df["pe"], higher_better=False) * FUND_WEIGHTS["pe"]
    )
    df["fund_score"] = fund_score * 100

    # IV score
    df["iv_rank"] = df["atm_iv"].rank(pct=True)
    iv_score = (
        score_series(df["atm_iv"], higher_better=True) * IV_WEIGHTS["atm_iv"] +
        score_series(df["iv_hv_ratio"], higher_better=True) * IV_WEIGHTS["iv_hv_ratio"] +
        score_series(df["term_slope"], higher_better=False) * IV_WEIGHTS["term_slope"] +
        score_series(df["iv_rank"], higher_better=True) * IV_WEIGHTS["iv_rank"]
    )
    df["iv_score"] = iv_score * 100

    # Composite
    df["total_score"] = FUNDAMENTAL_WEIGHT * df["fund_score"] + IV_WEIGHT * df["iv_score"]
    df["grade"] = df["total_score"].apply(assign_grade)

    df = df.sort_values("total_score", ascending=False)

    display(Markdown("---"))
    display(Markdown("## 🏆 IV Opportunity Scorecard"))

---

## 🏆 IV Opportunity Scorecard

In [18]:
if not df.empty:
    scorecard = df[[
        "ticker", "grade", "total_score", "fund_score", "iv_score",
        "spot", "sector", "atm_iv", "hv_30", "iv_hv_ratio", "term_slope", "trend_label"
    ]].head(TOP_N).copy()

    scorecard = scorecard.rename(columns={
        "ticker": "Ticker",
        "grade": "Grade",
        "total_score": "Total",
        "fund_score": "Fund",
        "iv_score": "IV Score",
        "spot": "Spot",
        "sector": "Sector",
        "atm_iv": "ATM IV",
        "hv_30": "HV 30d",
        "iv_hv_ratio": "IV/HV",
        "term_slope": "Term Slope",
        "trend_label": "Trend",
    })

    display_table(
        scorecard,
        caption="Top IV Opportunities (Ranked by Composite Score)",
        format_dict={
            "Total": "{:.1f}",
            "Fund": "{:.1f}",
            "IV Score": "{:.1f}",
            "Spot": "${:,.2f}",
            "ATM IV": "{:.1%}",
            "HV 30d": "{:.1%}",
            "IV/HV": "{:.2f}",
            "Term Slope": "{:.4f}",
        },
    )

,Ticker,Grade,Total,Fund,IV Score,Spot,Sector,ATM IV,HV 30d,IV/HV,Term Slope,Trend
3,MRVL,B,75.6,73.5,77.1,$80.12,Technology,65.1%,40.8%,1.59,0.0005,Side
7,WDC,C,69.2,72.5,67.1,$281.02,Technology,82.7%,94.4%,0.88,-0.0004,Up
6,ASX,D,57.6,42.7,67.5,$21.02,Technology,51.4%,34.9%,1.47,nan,Up
4,CCL,D,51.0,57.5,46.7,$33.49,Consumer Cyclical,47.9%,45.8%,1.05,0.0004,Up
1,F,F,48.8,35.2,57.9,$13.77,Consumer Cyclical,33.4%,26.1%,1.28,-0.0038,Up
5,NCLH,F,48.6,48.4,48.7,$23.06,Consumer Cyclical,57.0%,57.4%,0.99,0.0022,Side
0,AMZN,F,42.7,66.5,26.9,$208.51,Consumer Cyclical,31.4%,34.1%,0.92,-0.0003,Side
2,CARR,F,32.4,23.8,38.1,$64.15,Industrials,31.4%,26.3%,1.19,nan,Side


---
## 📊 Visualizations

In [19]:
if not df.empty:
    grade_colors = {
        "A+": "#15803d", "A": "#22c55e", "B": "#eab308",
        "C": "#f97316", "D": "#9ca3af", "F": "#dc2626", "N/A": "#6b7280",
    }

    fig1 = px.bar(
        df.head(TOP_N).sort_values("total_score", ascending=True),
        x="total_score",
        y="ticker",
        color="grade",
        title="Figure 1. IV Opportunity Composite Scores",
        labels={"total_score": "Score", "ticker": "Ticker"},
        color_discrete_map=grade_colors,
        orientation="h",
    )
    fig1.update_layout(height=500, xaxis_range=[0, 100])
    fig1.show()

In [20]:
if not df.empty:
    plot_df = df.dropna(subset=["atm_iv", "hv_30"])

    fig2 = px.scatter(
        plot_df,
        x="hv_30",
        y="atm_iv",
        size="total_score",
        color="iv_hv_ratio",
        text="ticker",
        color_continuous_scale="RdYlGn",
        title="Figure 2. ATM IV vs Historical Volatility",
        labels={"hv_30": "HV 30-day", "atm_iv": "ATM Implied Volatility"},
    )
    fig2.add_shape(
        type="line", x0=0, y0=0, x1=1, y1=1,
        line=dict(dash="dash", color="gray"),
    )
    fig2.update_traces(textposition="top center")
    fig2.update_layout(
        height=500,
        xaxis_tickformat=".0%",
        yaxis_tickformat=".0%",
    )
    fig2.show()

In [21]:
if not df.empty:
    sector_df = df.groupby("sector").agg({
        "atm_iv": "mean",
        "iv_hv_ratio": "mean",
        "total_score": "mean",
        "ticker": "count",
    }).reset_index().rename(columns={"ticker": "count"})

    fig3 = px.bar(
        sector_df.sort_values("atm_iv", ascending=True),
        x="atm_iv",
        y="sector",
        color="iv_hv_ratio",
        color_continuous_scale="Viridis",
        title="Figure 3. Average ATM IV by Sector",
        labels={"atm_iv": "Avg ATM IV", "sector": "Sector"},
        orientation="h",
    )
    fig3.update_layout(height=400, xaxis_tickformat=".0%")
    fig3.show()

---
## 🎯 Top Picks Summary

In [22]:
if not df.empty:
    display(Markdown("### 🥇 Top 5 IV Opportunities"))
    display(Markdown(""))

    for i, (_, row) in enumerate(df.head(5).iterrows(), 1):
        iv_pct = row['atm_iv'] * 100 if pd.notna(row['atm_iv']) else 0
        hv_pct = row['hv_30'] * 100 if pd.notna(row['hv_30']) else 0
        ratio = row['iv_hv_ratio'] if pd.notna(row['iv_hv_ratio']) else 0

        display(Markdown(
            f"**{i}. {row['ticker']}** ({row['sector']}) — "
            f"Grade **{row['grade']}**, Score {row['total_score']:.1f}\n\n"
            f"   - ATM IV: {iv_pct:.1f}% | HV: {hv_pct:.1f}% | IV/HV: {ratio:.2f} | Trend: {row['trend_label']}"
        ))

    display(Markdown("---"))
    display(Markdown("*Higher IV/HV ratio suggests IV premium over realized volatility — potential selling opportunity.*"))

### 🥇 Top 5 IV Opportunities

**1. MRVL** (Technology) — Grade **B**, Score 75.6

   - ATM IV: 65.1% | HV: 40.8% | IV/HV: 1.59 | Trend: Side

**2. WDC** (Technology) — Grade **C**, Score 69.2

   - ATM IV: 82.7% | HV: 94.4% | IV/HV: 0.88 | Trend: Up

**3. ASX** (Technology) — Grade **D**, Score 57.6

   - ATM IV: 51.4% | HV: 34.9% | IV/HV: 1.47 | Trend: Up

**4. CCL** (Consumer Cyclical) — Grade **D**, Score 51.0

   - ATM IV: 47.9% | HV: 45.8% | IV/HV: 1.05 | Trend: Up

**5. F** (Consumer Cyclical) — Grade **F**, Score 48.8

   - ATM IV: 33.4% | HV: 26.1% | IV/HV: 1.28 | Trend: Up

---

*Higher IV/HV ratio suggests IV premium over realized volatility — potential selling opportunity.*